In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

images = [cv2.imread('./images/andrew.jpeg'), cv2.imread('./images/bengio.jpeg'),
          cv2.imread('./images/goodfellow.jpeg'), cv2.imread('./images/leskovec.jpeg')]
note_image = cv2.imread('./note.png')

www = cv2.imread('./images/www.png')

In [2]:
def Convloution2D(img, filters, stride, padding):

    f = filters[0].shape[0]
    
    if (padding != 'valid') :  p = int((f - 1) / 2)  #get a padding size!
    else:
       p = 0

    # add pading to image
    new_image_size = (img.shape[0] + 2 * p, img.shape[1] + 2 * p)
    if img.ndim == 3: #number of dimension of an array -> if 3d
        image_padding = np.zeros((*new_image_size, img.shape[2]))  
    else:
        image_padding = np.zeros(new_image_size)

    for row in range(img.shape[0]):
        for col in range(img.shape[1]): 
            image_padding[row + p, col + p] = img[row, col]   #copy image field to image
    img = image_padding

    # finding output size images
    def out_size(img):
        X = int((img.shape[0] + 2 * (p - f)) / stride + 1)
        Y = int((img.shape[1] + 2 * (p - f)) / stride + 1)
        return (X, Y)
    output_size = out_size(img)
    outputs = [np.zeros(output_size) for fi in filters]

  
    # Convolution
    print(img.shape)
    for i, output in enumerate(outputs):
        for row in range(output_size[0]):
            for col in range(output_size[1]):
                output[row, col] = np.sum(
                    filters[i] * img[(row * stride):(row * stride) + f, (col * stride):(col * stride) + f])
        if img.ndim == 3:  #number of dimension of array
            output = output / img.shape[2]

    return outputs

SyntaxError: expected 'else' after 'if' expression (178837181.py, line 5)